In [20]:
import torch 
import numpy as np
from tqdm.notebook import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset = torch.load("../dataset.pt")

In [23]:
NC = []
AD = []
for data in dataset:
    if data[1] == 0:
        NC.append(data)
    else:
        AD.append(data)
        
NC = [sample[0] for sample in NC]
NCgan1 = [sample[0][0] for sample in NC]
NCgan2 = [sample[1][0] for sample in NC]
NCgan3 = [sample[2][0] for sample in NC]

705

In [18]:
import argparse
import os
import numpy as np
import math
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch


os.makedirs("images", exist_ok=True)

n_epochs = 200
batch_size = 64
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_cpu = 8
latent_dim = 100
n_classes = 10
img_size = 32
channels = 1
sample_interval = 400


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(n_classes, latent_dim)

        self.init_size = img_size // 4  # Initial size before upsampling
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise, labels):
        gen_input = torch.mul(self.label_emb(labels), noise)
        out = self.l1(gen_input)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            """Returns layers of each discriminator block"""
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.conv_blocks = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4

        # Output layers
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())
        self.aux_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, n_classes), nn.Softmax())

    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)

        return validity, label


# Loss functions
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()


    
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))


def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(torch.FloatTensor(np.random.normal(0, 1, (n_row ** 2, latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(torch.LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)


# ----------
#  Training
# ----------
img_list = []
G_losses = []
D_losses = []

for epoch in range(n_epochs):
    for i, (imgs, labels) in tqdm(enumerate(dataloader)):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(torch.FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(torch.FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(torch.FloatTensor))
        labels = Variable(labels.type(torch.LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(torch.FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
        gen_labels = Variable(torch.LongTensor(np.random.randint(0, n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity, pred_label = discriminator(gen_imgs)
        g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # Loss for fake images
        fake_pred, fake_aux = discriminator(gen_imgs.detach())
        d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        # Calculate discriminator accuracy
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        d_loss.backward()
        optimizer_D.step()
        
        
        G_losses.append(g_loss.item())
        D_losses.append(d_loss.item())
        
        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
        )
        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)



[Epoch 0/200] [Batch 0/938] [D loss: 1.497860, acc: 9%] [G loss: 1.499923]
[Epoch 0/200] [Batch 1/938] [D loss: 1.497961, acc: 10%] [G loss: 1.500127]
[Epoch 0/200] [Batch 2/938] [D loss: 1.497875, acc: 8%] [G loss: 1.499638]
[Epoch 0/200] [Batch 3/938] [D loss: 1.497920, acc: 7%] [G loss: 1.499563]
[Epoch 0/200] [Batch 4/938] [D loss: 1.497827, acc: 8%] [G loss: 1.499763]
[Epoch 0/200] [Batch 5/938] [D loss: 1.497803, acc: 5%] [G loss: 1.499290]
[Epoch 0/200] [Batch 6/938] [D loss: 1.497789, acc: 8%] [G loss: 1.499153]
[Epoch 0/200] [Batch 7/938] [D loss: 1.497477, acc: 9%] [G loss: 1.498912]
[Epoch 0/200] [Batch 8/938] [D loss: 1.497625, acc: 10%] [G loss: 1.498599]
[Epoch 0/200] [Batch 9/938] [D loss: 1.497760, acc: 11%] [G loss: 1.498834]
[Epoch 0/200] [Batch 10/938] [D loss: 1.497701, acc: 8%] [G loss: 1.498555]
[Epoch 0/200] [Batch 11/938] [D loss: 1.497640, acc: 9%] [G loss: 1.498878]
[Epoch 0/200] [Batch 12/938] [D loss: 1.497546, acc: 7%] [G loss: 1.498362]
[Epoch 0/200] [Batc

[Epoch 0/200] [Batch 107/938] [D loss: 1.496722, acc: 16%] [G loss: 1.495940]
[Epoch 0/200] [Batch 108/938] [D loss: 1.498589, acc: 12%] [G loss: 1.493736]
[Epoch 0/200] [Batch 109/938] [D loss: 1.496257, acc: 14%] [G loss: 1.494157]
[Epoch 0/200] [Batch 110/938] [D loss: 1.497172, acc: 11%] [G loss: 1.492554]
[Epoch 0/200] [Batch 111/938] [D loss: 1.497344, acc: 9%] [G loss: 1.491060]
[Epoch 0/200] [Batch 112/938] [D loss: 1.502101, acc: 14%] [G loss: 1.487896]
[Epoch 0/200] [Batch 113/938] [D loss: 1.499596, acc: 15%] [G loss: 1.489391]
[Epoch 0/200] [Batch 114/938] [D loss: 1.500716, acc: 14%] [G loss: 1.493989]
[Epoch 0/200] [Batch 115/938] [D loss: 1.499444, acc: 7%] [G loss: 1.495010]
[Epoch 0/200] [Batch 116/938] [D loss: 1.500271, acc: 17%] [G loss: 1.499821]
[Epoch 0/200] [Batch 117/938] [D loss: 1.498264, acc: 12%] [G loss: 1.502042]
[Epoch 0/200] [Batch 118/938] [D loss: 1.498849, acc: 14%] [G loss: 1.503022]
[Epoch 0/200] [Batch 119/938] [D loss: 1.498722, acc: 11%] [G loss

[Epoch 0/200] [Batch 213/938] [D loss: 1.497803, acc: 8%] [G loss: 1.467755]
[Epoch 0/200] [Batch 214/938] [D loss: 1.500051, acc: 20%] [G loss: 1.472639]
[Epoch 0/200] [Batch 215/938] [D loss: 1.502676, acc: 17%] [G loss: 1.477045]
[Epoch 0/200] [Batch 216/938] [D loss: 1.501378, acc: 8%] [G loss: 1.482611]
[Epoch 0/200] [Batch 217/938] [D loss: 1.500515, acc: 16%] [G loss: 1.488886]
[Epoch 0/200] [Batch 218/938] [D loss: 1.499779, acc: 22%] [G loss: 1.495503]
[Epoch 0/200] [Batch 219/938] [D loss: 1.497166, acc: 17%] [G loss: 1.505462]
[Epoch 0/200] [Batch 220/938] [D loss: 1.495031, acc: 13%] [G loss: 1.507755]
[Epoch 0/200] [Batch 221/938] [D loss: 1.496808, acc: 11%] [G loss: 1.509708]
[Epoch 0/200] [Batch 222/938] [D loss: 1.496734, acc: 15%] [G loss: 1.512741]
[Epoch 0/200] [Batch 223/938] [D loss: 1.496234, acc: 17%] [G loss: 1.514492]
[Epoch 0/200] [Batch 224/938] [D loss: 1.493544, acc: 17%] [G loss: 1.515964]
[Epoch 0/200] [Batch 225/938] [D loss: 1.496677, acc: 10%] [G loss

[Epoch 0/200] [Batch 319/938] [D loss: 1.412509, acc: 36%] [G loss: 1.463940]
[Epoch 0/200] [Batch 320/938] [D loss: 1.420415, acc: 32%] [G loss: 1.470851]
[Epoch 0/200] [Batch 321/938] [D loss: 1.436490, acc: 32%] [G loss: 1.453702]
[Epoch 0/200] [Batch 322/938] [D loss: 1.432673, acc: 28%] [G loss: 1.471565]
[Epoch 0/200] [Batch 323/938] [D loss: 1.427586, acc: 32%] [G loss: 1.484459]
[Epoch 0/200] [Batch 324/938] [D loss: 1.429157, acc: 32%] [G loss: 1.478375]
[Epoch 0/200] [Batch 325/938] [D loss: 1.417560, acc: 35%] [G loss: 1.453699]
[Epoch 0/200] [Batch 326/938] [D loss: 1.415200, acc: 36%] [G loss: 1.481020]
[Epoch 0/200] [Batch 327/938] [D loss: 1.424265, acc: 39%] [G loss: 1.457232]
[Epoch 0/200] [Batch 328/938] [D loss: 1.416892, acc: 35%] [G loss: 1.477532]
[Epoch 0/200] [Batch 329/938] [D loss: 1.398156, acc: 39%] [G loss: 1.490505]
[Epoch 0/200] [Batch 330/938] [D loss: 1.414720, acc: 39%] [G loss: 1.481516]
[Epoch 0/200] [Batch 331/938] [D loss: 1.378603, acc: 46%] [G lo

[Epoch 0/200] [Batch 425/938] [D loss: 1.302352, acc: 53%] [G loss: 1.468830]
[Epoch 0/200] [Batch 426/938] [D loss: 1.350518, acc: 46%] [G loss: 1.498071]
[Epoch 0/200] [Batch 427/938] [D loss: 1.345151, acc: 52%] [G loss: 1.408945]
[Epoch 0/200] [Batch 428/938] [D loss: 1.310450, acc: 52%] [G loss: 1.430921]
[Epoch 0/200] [Batch 429/938] [D loss: 1.283890, acc: 59%] [G loss: 1.425553]
[Epoch 0/200] [Batch 430/938] [D loss: 1.308175, acc: 50%] [G loss: 1.442699]
[Epoch 0/200] [Batch 431/938] [D loss: 1.309632, acc: 55%] [G loss: 1.373698]
[Epoch 0/200] [Batch 432/938] [D loss: 1.297606, acc: 56%] [G loss: 1.422570]
[Epoch 0/200] [Batch 433/938] [D loss: 1.289515, acc: 60%] [G loss: 1.369601]
[Epoch 0/200] [Batch 434/938] [D loss: 1.318810, acc: 53%] [G loss: 1.412051]
[Epoch 0/200] [Batch 435/938] [D loss: 1.292098, acc: 62%] [G loss: 1.408541]
[Epoch 0/200] [Batch 436/938] [D loss: 1.272146, acc: 63%] [G loss: 1.396607]
[Epoch 0/200] [Batch 437/938] [D loss: 1.318908, acc: 50%] [G lo

...

[Epoch 4/200] [Batch 107/938] [D loss: 1.107784, acc: 91%] [G loss: 1.143923]
[Epoch 4/200] [Batch 108/938] [D loss: 1.067514, acc: 95%] [G loss: 1.173959]
[Epoch 4/200] [Batch 109/938] [D loss: 1.076336, acc: 96%] [G loss: 1.143832]
[Epoch 4/200] [Batch 110/938] [D loss: 1.092389, acc: 94%] [G loss: 1.093866]
[Epoch 4/200] [Batch 111/938] [D loss: 1.082676, acc: 97%] [G loss: 1.183120]
[Epoch 4/200] [Batch 112/938] [D loss: 1.068938, acc: 97%] [G loss: 1.163622]
[Epoch 4/200] [Batch 113/938] [D loss: 1.136184, acc: 92%] [G loss: 1.206111]
[Epoch 4/200] [Batch 114/938] [D loss: 1.098047, acc: 94%] [G loss: 1.093589]
[Epoch 4/200] [Batch 115/938] [D loss: 1.096424, acc: 92%] [G loss: 1.162901]
[Epoch 4/200] [Batch 116/938] [D loss: 1.119993, acc: 92%] [G loss: 1.197501]
[Epoch 4/200] [Batch 117/938] [D loss: 1.063900, acc: 95%] [G loss: 1.128723]
[Epoch 4/200] [Batch 118/938] [D loss: 1.069776, acc: 96%] [G loss: 1.165712]
[Epoch 4/200] [Batch 119/938] [D loss: 1.072054, acc: 95%] [G lo

[Epoch 4/200] [Batch 213/938] [D loss: 1.079363, acc: 96%] [G loss: 1.112981]
[Epoch 4/200] [Batch 214/938] [D loss: 1.081783, acc: 93%] [G loss: 1.222423]
[Epoch 4/200] [Batch 215/938] [D loss: 1.079402, acc: 94%] [G loss: 1.162114]
[Epoch 4/200] [Batch 216/938] [D loss: 1.089114, acc: 94%] [G loss: 1.141466]
[Epoch 4/200] [Batch 217/938] [D loss: 1.095149, acc: 92%] [G loss: 1.180652]
[Epoch 4/200] [Batch 218/938] [D loss: 1.103521, acc: 97%] [G loss: 1.099407]
[Epoch 4/200] [Batch 219/938] [D loss: 1.089524, acc: 95%] [G loss: 1.168461]
[Epoch 4/200] [Batch 220/938] [D loss: 1.120011, acc: 93%] [G loss: 1.131237]
[Epoch 4/200] [Batch 221/938] [D loss: 1.088681, acc: 96%] [G loss: 1.091328]
[Epoch 4/200] [Batch 222/938] [D loss: 1.122661, acc: 93%] [G loss: 1.100647]
[Epoch 4/200] [Batch 223/938] [D loss: 1.059124, acc: 97%] [G loss: 1.116059]
[Epoch 4/200] [Batch 224/938] [D loss: 1.085222, acc: 92%] [G loss: 1.344396]
[Epoch 4/200] [Batch 225/938] [D loss: 1.060758, acc: 96%] [G lo

[Epoch 4/200] [Batch 319/938] [D loss: 1.052151, acc: 98%] [G loss: 1.207306]
[Epoch 4/200] [Batch 320/938] [D loss: 1.069471, acc: 94%] [G loss: 1.132184]
[Epoch 4/200] [Batch 321/938] [D loss: 1.100644, acc: 96%] [G loss: 1.144508]
[Epoch 4/200] [Batch 322/938] [D loss: 1.041316, acc: 97%] [G loss: 1.132318]
[Epoch 4/200] [Batch 323/938] [D loss: 1.070356, acc: 95%] [G loss: 1.175296]
[Epoch 4/200] [Batch 324/938] [D loss: 1.068794, acc: 93%] [G loss: 1.116359]
[Epoch 4/200] [Batch 325/938] [D loss: 1.079199, acc: 95%] [G loss: 1.138889]
[Epoch 4/200] [Batch 326/938] [D loss: 1.076421, acc: 92%] [G loss: 1.088183]
[Epoch 4/200] [Batch 327/938] [D loss: 1.084484, acc: 95%] [G loss: 1.117081]
[Epoch 4/200] [Batch 328/938] [D loss: 1.054626, acc: 94%] [G loss: 1.086707]
[Epoch 4/200] [Batch 329/938] [D loss: 1.103831, acc: 96%] [G loss: 1.150083]
[Epoch 4/200] [Batch 330/938] [D loss: 1.088156, acc: 93%] [G loss: 1.238668]
[Epoch 4/200] [Batch 331/938] [D loss: 1.069881, acc: 97%] [G lo

[Epoch 4/200] [Batch 425/938] [D loss: 1.079505, acc: 96%] [G loss: 1.158518]
[Epoch 4/200] [Batch 426/938] [D loss: 1.074367, acc: 98%] [G loss: 1.172156]
[Epoch 4/200] [Batch 427/938] [D loss: 1.103593, acc: 92%] [G loss: 1.154557]
[Epoch 4/200] [Batch 428/938] [D loss: 1.107198, acc: 96%] [G loss: 1.118174]
[Epoch 4/200] [Batch 429/938] [D loss: 1.081930, acc: 96%] [G loss: 1.087414]
[Epoch 4/200] [Batch 430/938] [D loss: 1.104280, acc: 92%] [G loss: 1.164209]
[Epoch 4/200] [Batch 431/938] [D loss: 1.139342, acc: 93%] [G loss: 1.172876]
[Epoch 4/200] [Batch 432/938] [D loss: 1.051584, acc: 96%] [G loss: 1.077385]
[Epoch 4/200] [Batch 433/938] [D loss: 1.070190, acc: 98%] [G loss: 1.137451]
[Epoch 4/200] [Batch 434/938] [D loss: 1.106651, acc: 95%] [G loss: 1.102748]
[Epoch 4/200] [Batch 435/938] [D loss: 1.081700, acc: 94%] [G loss: 1.126190]
[Epoch 4/200] [Batch 436/938] [D loss: 1.068545, acc: 96%] [G loss: 1.090822]
[Epoch 4/200] [Batch 437/938] [D loss: 1.113548, acc: 96%] [G lo

KeyboardInterrupt: 